In [4]:
#importamos todas las librerias necesarias
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
#from keras.layers import Dense, Conv2D, Deconvolution2D, MaxPooling2D, Flatten
from keras import layers as L
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, SeparableConv2D

Using TensorFlow backend.


In [5]:
#creamos la ruta donde se almacenan los datos del dataset
path = '../Euro_Sat/'

In [6]:
#Las imagenes del dataset serán los elementos de entrada, los elementos de salida serán a las clases a las que pertenecen esas fotos
dic = {}
entrada = []
salida = []
i = 0

for ele in os.listdir(path):
    if os.path.isdir(path+ele):
        for file in os.listdir(path+ele):
            im = Image.open(os.path.join(path,ele,file))
            im_array = np.array(im)
            entrada.append(im_array)
            salida.append(i)
        dic[i] = ele
        i += 1


In [7]:
#Se representan las clases mediante la codificación ‘one hot
clases_salida = to_categorical(salida)

In [8]:
#normalizamos las imágenes de modo que el rango de entrada a la CNN se encuentren entre 0 y 1
entrada = np.array(entrada)
entrada = entrada/255

In [9]:
#Creamos el conjunto de entrenamiento
x_train, x_test, y_train, y_test = train_test_split(entrada, clases_salida, test_size=0.2, shuffle=True)

In [10]:
#Creamos el modelo de la red neuronal
in_shape = x_train[0,:,:,:].shape

model = Sequential()

model.add(Conv2D(16, kernel_size=3, activation='relu', input_shape=in_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(50))
model.add(Dense(len(dic), activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 16)        448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)               

In [11]:
#Compilamos el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
#Entrenamos el modelo
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=25)

Train on 21600 samples, validate on 5400 samples
Epoch 1/25
21600/21600 [==============================] - 69s 3ms/step - loss: 1.1886 - accuracy: 0.5504 - val_loss: 0.7636 - val_accuracy: 0.7274
Epoch 2/25
21600/21600 [==============================] - 72s 3ms/step - loss: 0.7606 - accuracy: 0.7259 - val_loss: 0.6244 - val_accuracy: 0.7746
Epoch 3/25
21600/21600 [==============================] - 68s 3ms/step - loss: 0.6638 - accuracy: 0.7629 - val_loss: 0.6251 - val_accuracy: 0.7743
Epoch 4/25
21600/21600 [==============================] - 69s 3ms/step - loss: 0.5931 - accuracy: 0.7889 - val_loss: 0.5792 - val_accuracy: 0.7900
Epoch 5/25
21600/21600 [==============================] - 68s 3ms/step - loss: 0.5556 - accuracy: 0.8040 - val_loss: 0.5230 - val_accuracy: 0.8128
Epoch 6/25
21600/21600 [==============================] - 68s 3ms/step - loss: 0.4978 - accuracy: 0.8215 - val_loss: 0.6175 - val_accuracy: 0.7741
Epoch 7/25
21600/21600 [==============================] - 69s 3ms/ste

In [13]:
#Sacamos el accuracy del modelo
scores = model.evaluate(x_train, y_train, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 96.42%


In [55]:
#Guardamos el modelo para poder usarlo cuando se quiera
model_json = model.to_json()
with open("model1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model1.h5")
print("Saved model to disk")

Saved model to disk


In [14]:
#visualizamos las predicciones de varias muestras de prueba y las comparamos con las clases a las que realmente pertenecen.
prueba_test = model.predict(x_test)

In [15]:
for prueba in prueba_test[19]:
    print(round(prueba,2))

0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0


In [17]:
pred_test = [dic[np.argmax(p)] for p in prueba_test]

In [18]:
suelos_test = [dic[np.argmax(s)] for s in y_test]

In [19]:
s = suelos_test[50:60]

In [20]:
t = pred_test[50:60]

In [21]:
#Comparamos los datos de entrenamiento con los predecidos
import pandas as pd

d = {'y_Actual': s,
        'y_Predicted': t}

dat = pd.DataFrame(data=d)
dat


,y_Actual,y_Predicted
0,Industrial,Industrial
1,Highway,Highway
2,AnnualCrop,AnnualCrop
3,Forest,Forest
4,AnnualCrop,AnnualCrop
5,SeaLake,SeaLake
6,AnnualCrop,AnnualCrop
7,Forest,Forest
8,HerbaceousVegetation,HerbaceousVegetation
9,Forest,Forest


In [65]:
from sklearn.metrics import confusion_matrix
confusion_matrix(s, t)

array([[4, 0, 0, 1, 0, 0, 1, 0, 0, 0],
       [0, 5, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 7, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 5, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 2, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 3, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 4]], dtype=int64)

In [30]:
import matplotlib.pyplot as plt
my_image_resized = plt.imread("../Pruebas/test_sea.jpg")